In [3]:
# 📚 Basic libraries
import pandas as pd # data manipulation
import numpy as np # numerical operations
import warnings # warning messages management
# ⚙️ Settings
pd.set_option('display.max_columns', None) # display all columns
warnings.filterwarnings('ignore') # ignore warnings

# 🌐 New Libraries
from getpass import getpass # get password without showing it
import pymysql # MySQL connection
import requests # HTTP requests
import time # time operations

# ⚙️ Settings
pd.set_option('display.max_columns', None) # display all columns
warnings.filterwarnings('ignore') # ignore warnings

def create_connection(): # create a connection to a MySQL database
    """
    Create a connection to a MySQL database.
    Prompts the user for the database password and database name.
    
    Returns:
        cnx: pymysql.connections.Connection object
            The connection object to the MySQL database.
    """
    password = getpass.getpass("Please, kindly insert your password: ")
    database = input("Please, kindly insert your database name: ")
    cnx = pymysql.connect(user='root', password=password,
                          host='localhost', database=database)
    if cnx.open:
        print("Connection open")
    else:
        print("Connection is not successfully open")
    return cnx

In [5]:
# Pon tu contraseña y la base de datos
connection = create_connection()

Please, kindly insert your password:  ········
Please, kindly insert your database name:  sakila


Connection open


In [8]:
query = "SELECT * FROM actor;"  # Replace 'your_table_name' with an actual table name
df = pd.read_sql(query, connection)
df.head()

,actor_id,first_name,last_name,last_update
0,1,PENELOPE,GUINESS,2006-02-15 04:34:33
1,2,NICK,WAHLBERG,2006-02-15 04:34:33
2,3,ED,CHASE,2006-02-15 04:34:33
3,4,JENNIFER,DAVIS,2006-02-15 04:34:33
4,5,JOHNNY,LOLLOBRIGIDA,2006-02-15 04:34:33


In [30]:
def rentals_month(cnx, month, year):
    """
    Retrieve rental data for a given month and year.
    """
    query = f"""
    SELECT customer_id, rental_date
    FROM rental
    WHERE MONTH(rental_date) = {month} AND YEAR(rental_date) = {year}
    """
    df_rentals = pd.read_sql(query, cnx)
    return df_rentals


In [28]:
def rental_count_month(df_rentals, month, year):
    """
    Count the number of rentals made by each customer for a given month and year.
    """
    rental_count = df_rentals.groupby('customer_id').size().reset_index(name='rental_count')
    rental_count[f'rentals_{month:02d}_{year}'] = rental_count['rental_count']
    rental_count = rental_count.drop(columns=['rental_count'])
    return rental_count


In [26]:
def compare_rentals(df1, df2):
    """
    Compare the rental counts between two DataFrames and calculate the difference.
    """
    # Unir los DataFrames de mayo y junio por customer_id
    df_comparison = pd.merge(df1, df2, on='customer_id', how='inner')  # "inner" para clientes comunes en ambos meses
    
    # Calcular la diferencia entre las rentas
    df_comparison['difference'] = df_comparison[df2.columns[1]] - df_comparison[df1.columns[1]]
    
    return df_comparison

In [24]:
# Obtener datos de alquiler para mayo y junio de 2005
df_may = rentals_month(connection, 5, 2005)
df_june = rentals_month(connection, 6, 2005)

# Contar los alquileres por cliente en mayo y junio
df_may_count = rental_count_month(df_may, 5, 2005)
df_june_count = rental_count_month(df_june, 6, 2005)

# Comparar los alquileres de mayo y junio
df_comparison = compare_rentals(df_may_count, df_june_count)

# Mostrar el resultado
print(df_comparison)

     customer_id  rentals_05_2005  rentals_06_2005  difference
0              1              2.0              7.0         5.0
1              2              1.0              1.0         0.0
2              3              2.0              4.0         2.0
3              4              NaN              6.0         NaN
4              5              3.0              5.0         2.0
..           ...              ...              ...         ...
593          595              1.0              2.0         1.0
594          596              6.0              2.0        -4.0
595          597              2.0              3.0         1.0
596          598              NaN              1.0         NaN
597          599              1.0              4.0         3.0

[598 rows x 4 columns]
